In [3]:
from cStringIO import StringIO
import gzip
import os
import glob
import re
import pdb

import gcp
from gcp import storage
from gcp import bigquery as bq
import pandas as pd

# Import zip file from Google Cloud Storage
project = gcp.Context.default().project_id
bucket_name = project + '-datalab'
bucket_path = 'gs://' + bucket_name + '/data'
print 'Bucket: ' + bucket_path

Bucket: gs://datalab-projects-1331-datalab/data


In [4]:
# Downloading and extracting the files from cloud storage.

def process_datafile(localpath, storagepath, table, mode='create', overwrite=False):
  gzip_filename = 'citydata2.tar.gz'
  tar_filename = 'citydata2.tar'
  datadir = 'season_2'
  storagepath_r = storagepath.split('/')
  # If data has not been extracted, extract it.
  if not os.path.isdir(datadir):
    # If citydata.tar has not been downloaded, download it
    if not os.path.isfile(tar_filename):
      
      bucket_object = '/'.join(storagepath_r[:3]) + '/citydata2.tar.gz'
      %storage read --object $bucket_object --variable compressed_file
      
      gzip_file = gzip.GzipFile(fileobj=StringIO(compressed_file))
      del compressed_file

      import shutil
      with open(tar_filename, 'wb') as f_out:
        shutil.copyfileobj(gzip_file, f_out)

    import tarfile
    tar = tarfile.open(tar_filename, "r")
    tar.extractall()
    tar.close()
    os.remove(tar_filename)

  # Upload extracted file into GCS
  bucketname = storagepath_r[2]
  itempath = '/'.join(storagepath_r[3:])
  item = storage.Item(bucketname, itempath)
  if not item.exists() or overwrite:    
    with open(localpath, 'rb') as f:
      item.write_to(f.read(), 'text/plain')
  # Load data into Google BigQuery
  table.load(storagepath, mode=mode, csv_options=bq.CSVOptions(delimiter='\t'))

# Process Training Data

## Districts

In [5]:
table = bq.Table('datalab-projects-1331:xjk_algo_comp.districts')
if not table.exists():
  schema = bq.Schema([
      {'name': 'district_hash', 'type': 'STRING'},
      {'name': 'district_id', 'type': 'INTEGER'}
    ])
  table.create(schema)
  
  localpath = 'season_2/training_data/cluster_map/cluster_map'
  storagepath = os.path.join(bucket_path,localpath)
  process_datafile(localpath, storagepath, table)

## Weather

In [6]:
table = bq.Table('datalab-projects-1331:xjk_algo_comp.weather')
if not table.exists():
  schema = bq.Schema([
      {'name': 'time', 'type': 'STRING'},
      {'name': 'weather', 'type': 'INTEGER'},
      {'name': 'temperature', 'type': 'FLOAT'},
      {'name': 'pm25', 'type': 'FLOAT'}
    ])
  table.create(schema)
  
  wildpath = 'season_2/training_data/weather_data/*'
  for localpath in glob.glob(wildpath):
    storagepath = os.path.join(bucket_path,localpath)
    process_datafile(localpath, storagepath, table, mode='append')

In [7]:
%%bigquery udf --module transform_weather_time

/**
 * Pad with 0 or given string.
 *
 * @param int n Number to add padding to.
 * @param int width Width of number + padding.
 * @param string z (Optional) Other string to replace '0' as padding.
 */
function pad(n, width, z) {
  z = z || '0';
  n = n + '';
  return n.length >= width ? n : new Array(width - n.length + 1).join(z) + n;
}

/**
 * Transform timestamps of weather table into timeslots in weather table.
 *
 * @param {{time: string, weather: integer, temperature: float, pm25: float}} r
 * @param function({{time: string, weather: integer, temperature: float, pm25: float,
                     timeslot: string}}) emitFn
 */
function(r, emitFn) {
  var t = r.time.split(/[ :\-]/);
  var slot = Math.floor((parseInt(t[3]) * 60 + parseInt(t[4])) / 10) + 1;
  r.timeslot = t[0] + '-' + pad(t[1], 2) +
               '-' + pad(t[2], 2) + '-' + slot;
  emitFn(r);
}

In [8]:
%%bigquery execute -t datalab-projects-1331:xjk_algo_comp.weather -m overwrite
SELECT time, weather, temperature, pm25, timeslot
FROM transform_weather_time([datalab-projects-1331:xjk_algo_comp.weather])

time,weather,temperature,pm25,timeslot
2016-01-18 00:00:29,3,3.0,88.0,2016-01-18-1
2016-01-18 00:06:04,3,3.0,88.0,2016-01-18-1
2016-01-18 00:11:12,3,3.0,88.0,2016-01-18-2
2016-01-18 00:16:13,3,3.0,88.0,2016-01-18-2
2016-01-18 00:20:45,3,3.0,88.0,2016-01-18-3
2016-01-18 00:25:22,3,3.0,88.0,2016-01-18-3
2016-01-18 00:30:41,3,3.0,88.0,2016-01-18-4
2016-01-18 00:35:40,3,3.0,88.0,2016-01-18-4
2016-01-18 00:40:19,3,3.0,88.0,2016-01-18-5
2016-01-18 00:45:34,3,3.0,88.0,2016-01-18-5


## Traffic

In [9]:
table = bq.Table('datalab-projects-1331:xjk_algo_comp.traffic')
if not table.exists():
  schema = bq.Schema([
      {'name': 'district_hash', 'type': 'STRING'},
      {'name': 'tj_level1', 'type': 'INTEGER'},
      {'name': 'tj_level2', 'type': 'INTEGER'},
      {'name': 'tj_level3', 'type': 'INTEGER'},
      {'name': 'tj_level4', 'type': 'INTEGER'},
      {'name': 'tj_time', 'type': 'STRING'}
    ])
  table.create(schema)
  
  wildpath = 'season_2/training_data/traffic_data/*'
  for localpath in glob.glob(wildpath):
    with open(localpath, 'rb') as f:
      text = f.read()
    with open(localpath, 'wb') as f:
      f.write(re.sub(r'\b\t[0-9]:\b', '\t', text))
    storagepath = os.path.join(bucket_path,localpath)
    process_datafile(localpath, storagepath, table, mode='append', overwrite=True)

In [10]:
%%bigquery udf --module transform_traffic_time
  
/**
 * Pad with 0 or given string.
 *
 * @param int n Number to add padding to.
 * @param int width Width of number + padding.
 * @param string z (Optional) Other string to replace '0' as padding.
 */
function pad(n, width, z) {
  z = z || '0';
  n = n + '';
  return n.length >= width ? n : new Array(width - n.length + 1).join(z) + n;
}

/**
 * Transform timestamps of weather table into timeslots in traffic table.
 *
 * @param {{district_hash: string, tj_level1: integer, tj_level2: integer, tj_level3: integer, 
            tj_level4: integer, tj_time: string}} r
 * @param function({{district_hash: string, tj_level1: integer, tj_level2: integer, tj_level3: integer, 
                     tj_level4: integer, tj_time: string, timeslot: string}}) emitFn
 */
function(r, emitFn) {
  var t = r.tj_time.split(/[ :\-]/);
  var slot = Math.floor((parseInt(t[3]) * 60 + parseInt(t[4])) / 10) + 1;
  r.timeslot = t[0] + '-' + pad(t[1], 2) +
               '-' + pad(t[2], 2) + '-' + slot;
  emitFn(r);
}

In [11]:
%%bigquery execute -t datalab-projects-1331:xjk_algo_comp.traffic -m overwrite
  
SELECT district_hash, tj_level1, tj_level2, tj_level3, tj_level4, tj_time, timeslot
FROM transform_traffic_time([datalab-projects-1331:xjk_algo_comp.traffic])

district_hash,tj_level1,tj_level2,tj_level3,tj_level4,tj_time,timeslot
0a5fef95db34383403d11cb6af937309,46,1,1,0,2016-01-16 00:10:27,2016-01-16-2
0a5fef95db34383403d11cb6af937309,46,1,1,0,2016-01-16 00:10:27,2016-01-16-2
0a5fef95db34383403d11cb6af937309,46,1,1,0,2016-01-16 00:10:27,2016-01-16-2
0a5fef95db34383403d11cb6af937309,46,1,1,0,2016-01-16 00:10:27,2016-01-16-2
1c60154546102e6525f68cb4f31e0657,108,9,1,0,2016-01-16 00:10:27,2016-01-16-2
1c60154546102e6525f68cb4f31e0657,108,9,1,0,2016-01-16 00:10:27,2016-01-16-2
1c60154546102e6525f68cb4f31e0657,108,9,1,0,2016-01-16 00:10:27,2016-01-16-2
1c60154546102e6525f68cb4f31e0657,108,9,1,0,2016-01-16 00:10:27,2016-01-16-2
fff4e8465d1e12621bc361276b6217cf,121,5,3,0,2016-01-16 00:10:27,2016-01-16-2
fff4e8465d1e12621bc361276b6217cf,121,5,3,0,2016-01-16 00:10:27,2016-01-16-2


## POIs

In [12]:
table = bq.Table('datalab-projects-1331:xjk_algo_comp.pois')
  
localpath = 'season_2/training_data/poi_data/poi_data'

pois = []
pois_schema = [{'name': 'district_hash', 'type': 'STRING'}]
with open(localpath, 'rb') as f:
  for line in f:
    line_pois = map(lambda x: ['f{}'.format(x.split(':')[0].replace('#', '_')), x.split(':')[1]],
                    line.split('\t')[1:])
  for poi in line_pois:
    if poi[0] not in pois:
      pois.append(poi[0])
      pois_schema.append({'name': poi[0], 'type': 'INTEGER'})
pois.sort()
pois_schema = sorted(pois_schema, key=lambda k: k['name']) 

if not table.exists():
  schema = bq.Schema.from_data(pois_schema)
  table.create(schema)
  
  pois_data = pd.DataFrame(columns=['district_hash'] + pois)
  with open(localpath, 'rb') as f:
    for line in f:
      hash_pois = {}
      for poi_line in line.split('\t')[1:]:
        hash_pois['f{}'.format(poi_line.split(':')[0].replace('#', '_'))] = poi_line.split(':')[1]
      poi_data = [line.split('\t')[0]]
      # hash_pois = {f1_1: 15, ...}
      # pois = ['f1_1', ...]
      for poi in pois:
        value = '0'
        if poi in hash_pois:
          value = hash_pois[poi].strip()
        poi_data.append(value)
      pois_data.loc[len(pois_data)] = poi_data
  for poi in pois:
    pois_data[poi] = pd.to_numeric(pois_data[poi])
  table.insert_data(pois_data)

IOError: [Errno 2] No such file or directory: 'season_2/training_data/poi_data/poi_data'

Following code can be used to print out feature fields to be used when selecting from tables.

In [ ]:
final_text = ''
for counter, poi_text in enumerate(map(lambda x: 'FIRST(pois.{}) AS {}'.format(x,x), pois)):
  if counter%3 == 0:
    final_text = '{}\n'.format(final_text)
  final_text = '{}{}, '.format(final_text, poi_text)
print final_text[1:(len(final_text)-2)]

## Orders

In [ ]:
table = bq.Table('datalab-projects-1331:xjk_algo_comp.orders')
if not table.exists():
  schema = bq.Schema([{'name': 'order_id', 'type': 'STRING'},
                      {'name': 'driver_id', 'type': 'STRING'},
                      {'name': 'passenger_id', 'type': 'STRING'},
                      {'name': 'start_district_hash', 'type': 'STRING'},
                      {'name': 'dest_district_hash', 'type': 'STRING'},
                      {'name': 'price', 'type': 'FLOAT'},
                      {'name': 'time', 'type': 'STRING'}])
  table.create(schema)
  
  wildpath = 'season_2/training_data/order_data/*'
  for localpath in glob.glob(wildpath):
    print 'loading {}'.format(localpath)
    storagepath = os.path.join(bucket_path,localpath)
    process_datafile(localpath, storagepath, table, mode='append')